### This notebook illustrates how to generate samples from conditions
For illustration, we use the first 10 samples in the test slices as conditions to generate 10 accompaniments. You can also define your own conditions

In [1]:
import sys
import os
import torch
from model import init_ldm_model
from model.model_sdf import Diffpro_SDF
from model.sampler_sdf import SDFSampler
from generation_utils.fine_grained_control import X0EditFunc

import numpy as np
import pickle
from train.train_params import params_combined_cond, params_separate_cond
from data.prepare_training_pianoroll.convert_to_midi import extend_piano_roll, piano_roll_to_midi, save_midi
from generation_utils.fine_grained_control import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### set the global parameters here

In [2]:
''' use this group if you want to generate accompaniment conditioning on chord and melody'''
SEPARATE_MELODY_ACCOMPANIMENT = True # True if want to use a model that generates accompaniment conditioning on chord and melody; False if want to train a model that generates melody and accompaniment conditioning on chord
MODEL_PATH = 'results/model-separate_melody_accompaniment-/example/chkpts/weights_example.pt' # path of the model
CONDITION_DATA_PATH = 'data/train_test_slices/test_slices_separate_melody_accompaniment.pkl' # path of the condition data
SAVE_CHORD_IN_MIDI = False # whether to save the chords in midi file

''' use this group if you want to generate accompaniment and melody conditioning on chord'''
# SEPARATE_MELODY_ACCOMPANIMENT = False # True if want to use a model that generates accompaniment conditioning on chord and melody; False if want to train a model that generates melody and accompaniment conditioning on chord
# MODEL_PATH = 'results/model-combine_melody_accompaniment-/05-25_013713/chkpts/weights_best.pt' # path of the model
# CONDITION_DATA_PATH = 'data/train_test_slices/test_slices_combine_melody_accompaniment.pkl' # path of the condition data
# SAVE_CHORD_IN_MIDI = False # whether to save the chords in midi file

' use this group if you want to generate accompaniment and melody conditioning on chord'

#### start generation

In [3]:
# load the model
if SEPARATE_MELODY_ACCOMPANIMENT:
    ldm_model = init_ldm_model(params_separate_cond, debug_mode=False)
else:
    ldm_model = init_ldm_model(params_combined_cond, debug_mode=False)
model = Diffpro_SDF.load_trained(ldm_model, MODEL_PATH).to(device)
sampler = SDFSampler(model.ldm, 64, 64, is_autocast=False, device=device, debug_mode=False)


# Load condition dataset
with open(CONDITION_DATA_PATH, 'rb') as f:
    test_data = pickle.load(f)

print(test_data.shape)

/home/music/chord_trainer_clean/model/model_sdf.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_leaner = torch.load(chkpt_fpath, map_location=device)
/home/musi

(21204, 8, 64, 64)


In [4]:
background_cond = test_data[0:10,2:,:,:].copy() # use the sample with index 0-9 for generation
background_cond = torch.Tensor(background_cond).to(device)

output_x = sampler.generate(background_cond=background_cond, batch_size=background_cond.shape[0], 
                            same_noise_all_measure=False, X0EditFunc=X0EditFunc, use_classifier_free_guidance=True,
                            use_melody=SEPARATE_MELODY_ACCOMPANIMENT, device=device.type)
output_x = torch.clamp(output_x, min=0, max=1)
output_x = output_x.cpu().numpy()

[10, 2, 64, 64]


p_sample
editing
editing rhythm
p_sample
editing
editing rhythm
p_sample
editing
editing rhythm
p_sample
editing
editing rhythm
p_sample
editing
editing rhythm
p_sample
editing
editing rhythm
p_sample
editing
editing rhythm
p_sample
editing
editing rhythm
p_sample
editing
editing rhythm
p_sample
editing
editing rhythm


In [5]:
# save into midi files
import subprocess

for i in range(output_x.shape[0]):
    full_roll = extend_piano_roll(output_x[i]) # accompaniment roll
    full_melody_roll = None # melody roll
    if background_cond.shape[1]>=6:
        if background_cond[:,4:6,:,:].min()>=0:
            full_melody_roll = extend_piano_roll(background_cond[i,4:6,:,:].cpu().numpy())

    midi_file = piano_roll_to_midi(full_roll, None, full_melody_roll, bpm=80)
    os.makedirs('generated_samples/from_test_slices', exist_ok=True)
    filename = f"generated_samples/from_test_slices/sample_{i}.mid"
    save_midi(midi_file, filename)

    # convert midi to wav
    subprocess.Popen(['timidity',f"generated_samples/from_test_slices/sample_{i}.mid",'-Ow','-o',f"generated_samples/from_test_slices/sample_{i}.wav"]).communicate()

Playing generated_samples/from_test_slices/sample_0.mid
MIDI file: generated_samples/from_test_slices/sample_0.mid
Format: 1  Tracks: 3  Divisions: 220
Playing time: ~16 seconds
Notes cut: 0
Notes lost totally: 0


Playing generated_samples/from_test_slices/sample_1.mid
MIDI file: generated_samples/from_test_slices/sample_1.mid
Format: 1  Tracks: 3  Divisions: 220
Playing time: ~16 seconds
Notes cut: 0
Notes lost totally: 0
Playing generated_samples/from_test_slices/sample_2.mid
MIDI file: generated_samples/from_test_slices/sample_2.mid
Format: 1  Tracks: 3  Divisions: 220
Playing time: ~16 seconds
Notes cut: 0
Notes lost totally: 0
Playing generated_samples/from_test_slices/sample_3.mid
MIDI file: generated_samples/from_test_slices/sample_3.mid
Format: 1  Tracks: 3  Divisions: 220
Playing time: ~16 seconds
Notes cut: 0
Notes lost totally: 0
Playing generated_samples/from_test_slices/sample_4.mid
MIDI file: generated_samples/from_test_slices/sample_4.mid
Format: 1  Tracks: 3  Divisions: 220
Playing time: ~16 seconds
Notes cut: 0
Notes lost totally: 0
Playing generated_samples/from_test_slices/sample_5.mid
MIDI file: generated_samples/from_test_slices/sample_5.mid
Format: 1  Tracks: 3  Divisions: 